In [67]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [122]:
from matplotlib import pyplot as plt

In [140]:
from dist import RawJointDist, CPT
from rv import Variable, binvar, Unit
import numpy as np
from pdg import PDG
np.set_printoptions(precision=3)



In [141]:
A = Variable.alph("A", 3)
R = binvar("R")
Q = binvar("Q")


In [210]:
M = PDG()
M += {"cpd": CPT.from_matrix(A, R, [[.2, .8], [.3, .7], [.4, .6]]), 'alpha' : 0, "beta" : 1000}
M += Q,A,R
#M += {"from":"A", "to":"Q", "α":1}
M += "1 -> Q (α=1;β=0)"
# M += "1 -> A (α=0.8;β=0)"

M += {'cpd': CPT.from_matrix(R, Q, [[1,0], [np.nan,np.nan]]), 'α': 0}
# M.draw()
# M.varlist

('α', 1.0)


In [211]:
μ1 = M.factor_product()
μ2 = M.optimize_score(gamma=1)
μ3 = M.optimize_score(gamma=0.001)

In [202]:
μ2[Q]

,q,~q
⋆,0.901329,0.098671


In [175]:
[*M.edges('α')]

[0, 0.8, 0.8, 0]

In [239]:
def testdbN(nAs = 3):
    A = Variable.alph("A", nAs)
    R = binvar("R")
    Q = binvar("Q")

    M = PDG()
    M += {"cpd": CPT.from_matrix(A, R, [[.3, 7]]*nAs), 'alpha' : 0}
    M += Q,A,R
    #M += {"from":"A", "to":"Q", "α":1}
    M += "A -> Q (α=0.8;β=0)"
    M += "1 -> A (α=0.8;β=0)"
    M += {'cpd': CPT.from_matrix(R, Q, [[1,0], [np.nan,np.nan]]), 'α': 0}
    return M.optimize_score(gamma=0.5)

[0, 0.5, 0]

# Test Lax PDG scoring

Some setup...

In [ ]:
np.set_printoptions(precision=3)
from functools import partial

M2 = PDG()
X = binvar('X'); Y = binvar('Y')
M2 += CPT.from_matrix(X,Y, [[1,0], [np.nan, np.nan]])
dists = [
    RawJointDist(np.array([[1,0],[0,0]]), M2.varlist),
    RawJointDist(np.array([[.5,0],[.25,.25]]), M2.varlist),
    RawJointDist(np.array([[.5,0],[.5,0]]), M2.varlist),
    RawJointDist(np.array([[.3333,0],[.3333,.3333]]), M2.varlist),
    RawJointDist(np.array([[.25,.25],[.25,.25]]), M2.varlist)
]

In [ ]:
def fmt(num): 
#    return num
    if num.real == num: 
        return "%.3f" % num.real
    return "%.2f+%.1fj" % (num.real, num.imag)

def printM2scores(alpha,gamma):  
    def M2fastscore(rjd):
        return M2._build_fast_scorer(gamma=gamma)(rjd.data.reshape(-1))[0]
    
    M2.edgedata[('X', 'Y', 0)]['alpha'] = alpha
    for score in [partial(M2.score,gamma=gamma), M2fastscore]:
        print("score", *[fmt(score(d)) for d in dists],sep=' \t')
    print("Inc", *[fmt(M2.Inc(d)) for d in dists],sep= ' \t')
    print('IDef',  *[fmt(M2.IDef(d)) for d in dists],sep=' \t')
    print('μ*', M2.optimize_score(gamma).data)

Let's do $\gamma=0$, which is the same for both $\alpha=0$ and $\alpha=1$.


In [ ]:
np.set_printoptions(precision=3)
printM2scores(0,0)

So, as expected, when $\gamma=0$, only Inc matters, and so the most important thing is that the upper left number is close to zero. I'm not sure why the bottom numbers are also small, but it could be because the gradient is either messed up.

## Now, to see the effect of $\alpha$.
In both cases, we do this by setting $\gamma=1$. Since $\alpha=1$ is default (for the edge $X \to Y$), we have:

In [ ]:
printM2scores(1,1)

This makes sense: the important thing is to maximize the entropy of $X$; and so the middle two get the same score (and the $\frac13$-$\frac13$ split puts less entropy on X ). Since we start from a uniform distribution, the computed optimum is option 2.

-------

Now, we set $\alpha:= 0$ and see what happens.

In [ ]:
printM2scores(0,1)